In [26]:
import pandas as pd

In [27]:
import sqlite3

# Connect to SQLite database (creates a new database if it doesn't exist)
conn = sqlite3.connect('example2.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# SQL command to create a table
create_table_query = '''
CREATE TABLE IF NOT EXISTS user_profile_data (
    User_id INTEGER ,
   Install_Date DATETIME,
   Role STRING,
   Org_ID VARCHAR
);
'''

# Execute the SQL command to create the table
cursor.execute(create_table_query)




In [28]:
user_data = [
    (1,"2022-05-01 12:30:45", "Site Engineer","O1"),
(1,"2023-04-01 00:30:46","Planning Manager","O2"),
(1,"2023-07-01 05:45", "Owner","OAE1"),
(2,"2021-04-03 14:23","SIte Engineer","OO1"),
(2,"2022-12-12 03:25","Site Supervisor","OO2"),
(3,"2023-01-01 16:45","Project Manager","AA1"),
(3,"2023-03-03 10:10","Project Manager","AA2"),
(3,"2023-03-03 11:11","CXO","AA3" )
]

In [29]:
insert_query = '''
INSERT INTO user_profile_data (User_id, Install_Date,Role,Org_ID) VALUES (?, ?, ?, ?)
'''

# Execute the SQL command for each set of data
cursor.executemany(insert_query, user_data)

In [30]:
cursor.execute("SELECT * FROM user_profile_data")

In [31]:
result = pd.read_sql_query("SELECT * FROM user_profile_data",conn)
print(result)


   User_id         Install_Date              Role Org_ID
0        1  2022-05-01 12:30:45     Site Engineer     O1
1        1  2023-04-01 00:30:46  Planning Manager     O2
2        1     2023-07-01 05:45             Owner   OAE1
3        2     2021-04-03 14:23     SIte Engineer    OO1
4        2     2022-12-12 03:25   Site Supervisor    OO2
5        3     2023-01-01 16:45   Project Manager    AA1
6        3     2023-03-03 10:10   Project Manager    AA2
7        3     2023-03-03 11:11               CXO    AA3


In [32]:
#Write a query to return the latest entry against the user id using SubQuery & Join.


query1="""
SELECT u.user_id, u.install_date, u.role, u.org_id
FROM user_profile_data u
JOIN (
    SELECT user_id, MAX(install_date) AS max_install_date
    FROM user_profile_data
    GROUP BY user_id
) e ON u.user_id = e.user_id AND u.install_date = e.max_install_date;

"""

In [33]:
result = pd.read_sql_query(query1,conn)
print(result)


   User_id      Install_Date             Role Org_ID
0        1  2023-07-01 05:45            Owner   OAE1
1        2  2022-12-12 03:25  Site Supervisor    OO2
2        3  2023-03-03 11:11              CXO    AA3


In [34]:
#
query2="""
SELECT user_id, install_date, role, org_id
FROM (
SELECT user_id, install_date, role, org_id,
ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY install_date DESC) AS row_num
FROM user_profile_data
) AS ranked_entries
WHERE row_num = 1;

"""

In [35]:
result = pd.read_sql_query(query2,conn)
print(result)


   user_id      install_date             role org_id
0        1  2023-07-01 05:45            Owner   OAE1
1        2  2022-12-12 03:25  Site Supervisor    OO2
2        3  2023-03-03 11:11              CXO    AA3


In [36]:
user_data_2=[
    ("56KHB",1234,"2021-01-20 13:33:44","biscuit","food",100),
    ("56KHB",1234,"2021-01-20 13:33:44","crocin","medicine",50),
    ("32HBK",1235,"2021-08-20 13:38:55","chips","food",100),
    ("67ABC",1236,"2021-08-20 15:32:12","shoes","footwear",1200),
    ("67ABC",1236,"2021-08-20 15:32:12","shirt","clothing",500),
    ("67ABC",1236,"2021-01-20 15:32:12","earphones","electronics",450),
    ("67ABC",1237,"2021-08-20 16:18:19","laptop","electronics",45000),
    ("67ABC",1237,"2021-08-20 16:18:19","socks","clothing",150)
]

In [37]:
create_table_query2= '''
CREATE TABLE IF NOT EXISTS order_table (user_id VARCHAR,
order_id INTEGER,
purchase_datetime DATETIME,
product VARCHAR,
category VARCHAR,
product_revenue INTEGER
);
'''

In [38]:
cursor.execute(create_table_query2)


In [39]:
insert_query = '''
INSERT INTO order_table (User_id,order_id,purchase_datetime,product,category,product_revenue) VALUES (?, ?, ?, ?,?,?)
'''

# Execute the SQL command for each set of data
cursor.executemany(insert_query, user_data_2)

In [40]:
result = pd.read_sql_query("SELECT * FROM order_table",conn)
print(result)

  user_id  order_id    purchase_datetime    product     category  \
0   56KHB      1234  2021-01-20 13:33:44    biscuit         food   
1   56KHB      1234  2021-01-20 13:33:44     crocin     medicine   
2   32HBK      1235  2021-08-20 13:38:55      chips         food   
3   67ABC      1236  2021-08-20 15:32:12      shoes     footwear   
4   67ABC      1236  2021-08-20 15:32:12      shirt     clothing   
5   67ABC      1236  2021-01-20 15:32:12  earphones  electronics   
6   67ABC      1237  2021-08-20 16:18:19     laptop  electronics   
7   67ABC      1237  2021-08-20 16:18:19      socks     clothing   

   product_revenue  
0              100  
1               50  
2              100  
3             1200  
4              500  
5              450  
6            45000  
7              150  


In [41]:
query21="""
SELECT
COUNT(product)
FROM order_table
WHERE strftime('%Y', purchase_datetime)="2021" AND strftime('%m', purchase_datetime)="01"


"""

In [42]:
#1.Write a query to find the number of products bought in the month of January 2021.
result = pd.read_sql_query(query21,conn)
print(result)


   COUNT(product)
0               3


In [43]:
#2.Write a query to find the second order_id for each user_id ( Without Window Function)

query22="""
SELECT user_id, 
MIN(order_id) AS second_order_id
FROM order_table AS ot1
WHERE order_id > (
SELECT MIN(order_id)
FROM order_table AS ot2
WHERE ot1.user_id = ot2.user_id
)
GROUP BY user_id;
"""

In [44]:
result = pd.read_sql_query(query22,conn)
print(result)

  user_id  second_order_id
0   67ABC             1237


In [45]:
#3. Write a query to find the min, max and average time between two orders for any user.	
query23="""
SELECT user_id,
       MIN(time_diff) AS min_time_between_orders,
       MAX(time_diff) AS max_time_between_orders,
       AVG(time_diff) AS avg_time_between_orders
FROM (
    SELECT user_id,
           purchase_datetime,
           (SELECT MIN(purchase_datetime) 
            FROM order_table 
            WHERE user_id = o.user_id AND purchase_datetime > o.purchase_datetime) AS next_purchase_datetime,
           CAST((julianday((SELECT MIN(purchase_datetime) 
                            FROM order_table 
                            WHERE user_id = o.user_id AND purchase_datetime > o.purchase_datetime)) - julianday(purchase_datetime)) * 24 * 60 AS INTEGER) AS time_diff
    FROM order_table AS o
) AS time_diffs
WHERE next_purchase_datetime IS NOT NULL
GROUP BY user_id;

"""

In [46]:
result = pd.read_sql_query(query23,conn)
print(result)

  user_id  min_time_between_orders  max_time_between_orders  \
0   67ABC                       46                   305280   

   avg_time_between_orders  
0            101790.666667  


In [47]:
import pandas as pd


mobile_data = {"Name": "xyz",
"Mobile": "Iphone",
    "Mobileid": 23,
    "Mapping": {
        "Imei": 12344567777,
        "Specification": {
            "Model": "Apple 12",
            "OS Version": 17.0,
        },
        "Country": "us",
        "Product_name": "iphone 12"
    }
}


output_data = {
    "Imei": mobile_data["Mapping"]["Imei"],
    "Model": mobile_data["Mapping"]["Specification"]["Model"],
    "OS Version": mobile_data["Mapping"]["Specification"]["OS Version"],
    "Country": mobile_data["Mapping"]["Country"],
    "Product_name": mobile_data["Mapping"]["Product_name"],
    "Mobileid": mobile_data["Mobileid"]
}


output_df = pd.DataFrame([output_data])


print(output_df)


          Imei     Model  OS Version Country Product_name  Mobileid
0  12344567777  Apple 12        17.0      us    iphone 12        23
